In [1]:
import sys
import os
import time
import uuid
import cv2
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import albumentations as alb

In [2]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [4]:
cap = cv2.VideoCapture(1)

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


In [8]:
os.makedirs(IMAGES_PATH, exist_ok=True)
for imgnum in range(number_images):
    ret,frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname,frame)
    cv2.imshow('nihad',frame)
    time.sleep(0.5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
print('done')

done


In [ ]:
!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: /Users/muhammednihad/.labelmerc
2023-08-15 11:33:14.445 python[25920:2307744] +[CATransaction synchronize] called within transaction
2023-08-15 11:33:37.979 python[25920:2307744] +[CATransaction synchronize] called within transaction
2023-08-15 11:33:48.221 python[25920:2307744] +[CATransaction synchronize] called within transaction
2023-08-15 11:33:50.625 python[25920:2307744] +[CATransaction synchronize] called within transaction


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
images = tf.data.Dataset.list_files('data/images/*.jpg', shuffle=False)

2023-08-15 11:54:48.890693: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-08-15 11:54:48.890784: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-08-15 11:54:48.890796: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-08-15 11:54:48.890982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-15 11:54:48.891285: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
images.as_numpy_iterator().next()

2023-08-15 11:54:52.110071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [90]
	 [[{{node Placeholder/_0}}]]


b'data/images/40b3ecea-3b31-11ee-9c10-fa67b53f988b.jpg'

In [6]:
total_samples = len(list(images))

# Calculate the sizes of the different sets
train_size = int(0.7 * total_samples)  # 70% for training
cv_size = int(0.15 * total_samples)    # 15% for cross-validation
test_size = total_samples - train_size - cv_size  # Remaining for testing

# Shuffle the dataset if needed
images = images.shuffle(total_samples, reshuffle_each_iteration=False)

# Split the dataset into train, CV, and test sets
train_dataset = images.take(train_size)
remaining_dataset = images.skip(train_size)
cv_dataset = remaining_dataset.take(cv_size)
test_dataset = remaining_dataset.skip(cv_size)

In [7]:
import os
import shutil
import random

# Set the paths for the source images and destination folders
source_images_path = 'data/images/'
destination_train_path = 'data/train/images'
destination_cv_path = 'data/val/images'
destination_test_path = 'data/test/images'

# Create the destination directories if they don't exist
os.makedirs(destination_train_path, exist_ok=True)
os.makedirs(destination_cv_path, exist_ok=True)
os.makedirs(destination_test_path, exist_ok=True)

# Get a list of all image file paths
image_file_paths = [image.decode('utf-8') for image in list(images.as_numpy_iterator())]

# Shuffle the list to ensure randomness
# random.shuffle(image_file_paths)

# Calculate the number of images for each set
total_images = len(image_file_paths)
train_count = int(0.7 * total_images)
cv_count = int(0.15 * total_images)
test_count = total_images - train_count - cv_count
print(train_count,cv_count,test_count)
# Copy images to respective folders
for i, image_path in enumerate(image_file_paths):
    if i < train_count:
        destination_folder = destination_train_path
    elif i < train_count + cv_count:
        destination_folder = destination_cv_path
    else:
        destination_folder = destination_test_path
    
    destination_file = os.path.join(destination_folder, os.path.basename(image_path))
    shutil.copy(image_path, destination_file)

print("Images copied to respective folders.")


62 13 15
Images copied to respective folders.


2023-08-15 11:55:37.798803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [90]
	 [[{{node Placeholder/_0}}]]
2023-08-15 11:55:37.798999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [90]
	 [[{{node Placeholder/_0}}]]


In [9]:
for folder in ['train', 'test', 'val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        filename = file.split('.')[0] + '.json'
        existing_filepath = os.path.join('data', 'labels', filename)

        if os.path.exists(existing_filepath):
            new_folder = os.path.join('data', folder, 'labels')
            os.makedirs(new_folder, exist_ok=True)  # Create folder if it doesn't exist
            new_filepath = os.path.join(new_folder, filename)
            shutil.copy(existing_filepath, new_filepath)

print("JSON label files copied to respective folders.")

JSON label files copied to respective folders.


In [10]:
len(train_dataset)

62

In [11]:
len(cv_dataset)

13

In [12]:
len(test_dataset)

15

In [13]:
augmentor = alb.Compose([
                        alb.HorizontalFlip(p=0.5),
                        alb.RandomBrightnessContrast(p=0.2),
                        alb.RandomGamma(p=0.2),
                        alb.RGBShift(p=0.2),
                        alb.VerticalFlip(p=0.5),
                        ],
                       bbox_params = alb.BboxParams(format='albumentations',
                                                   label_fields=['class_labels']))

In [32]:
img = cv2.imread(os.path.join('data','train', 'images','4e6e59ec-3b31-11ee-9c10-fa67b53f988b.jpg'))

In [34]:
img.shape

(1080, 1920, 3)

In [26]:
with open(os.path.join('data', 'train', 'labels', '4e6e59ec-3b31-11ee-9c10-fa67b53f988b.json'), 'r') as f:
    label = json.load(f)

In [27]:
label['shapes'][0]['points']

[[756.1538461538462, 94.61538461538461],
 [1394.6153846153845, 905.3846153846154]]

In [28]:
coords = [0,0,0,0]
coords[0] = label['shapes'][0]['points'][0][0]
coords[1] = label['shapes'][0]['points'][0][1]
coords[2] = label['shapes'][0]['points'][1][0]
coords[3] = label['shapes'][0]['points'][1][1]

In [29]:
coords

[756.1538461538462, 94.61538461538461, 1394.6153846153845, 905.3846153846154]

In [35]:
coords = list(np.divide(coords, [1080,1920,1080,1920])) 

In [36]:
coords

[0.0010939725783475783,
 0.00010266426282051282,
 0.002017672720797721,
 0.0009824051816239316]

In [15]:
for partition in ['train', 'test', 'val']:
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0, 0, 0.00001, 0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640, 480, 640, 480]))

        try:
            # Create the destination folder if it does not exist
            destination_folder = os.path.join('aug_data', partition, 'images')
            os.makedirs(destination_folder, exist_ok=True)

            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                output_image_path = os.path.join(destination_folder, f'{image.split(".")[0]}.{x}.jpg')
                cv2.imwrite(output_image_path, augmented['image'])

                annotation = {'image': image}

                if os.path.exists(label_path) and len(augmented['bboxes']) > 0:
                    annotation['bbox'] = augmented['bboxes'][0]
                    annotation['class'] = 1
                else:
                    annotation['bbox'] = [0, 0, 0, 0]
                    annotation['class'] = 0

                output_label_path = os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json')
                os.makedirs(os.path.dirname(output_label_path), exist_ok=True)
                with open(output_label_path, 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

Expected x_min for bbox [1.157451923076923, 0.32211538461538464, 2.0084134615384612, 1.8637820512820513, 'face'] to be in the range [0.0, 1.0], got 1.157451923076923.
Expected x_max for bbox [0.7560096153846154, 0.8766025641025641, 1.7007211538461537, 1.7900641025641024, 'face'] to be in the range [0.0, 1.0], got 1.7007211538461537.
Expected x_min for bbox [1.0733173076923077, 0.3189102564102564, 1.9939903846153844, 1.6810897435897436, 'face'] to be in the range [0.0, 1.0], got 1.0733173076923077.
Expected x_max for bbox [0.09014423076923075, 0.9086538461538463, 1.2055288461538463, 2.0176282051282053, 'face'] to be in the range [0.0, 1.0], got 1.2055288461538463.
Expected x_min for bbox [1.3954326923076923, 0.5560897435897435, 2.0685096153846154, 1.780448717948718, 'face'] to be in the range [0.0, 1.0], got 1.3954326923076923.
Expected x_max for bbox [0.9098557692307692, 0.33814102564102566, 2.1237980769230766, 2.004807692307692, 'face'] to be in the range [0.0, 1.0], got 2.12379807692

Expected x_min for bbox [1.532451923076923, 0.4471153846153847, 2.1646634615384612, 1.7451923076923075, 'face'] to be in the range [0.0, 1.0], got 1.532451923076923.
Expected x_min for bbox [1.0204326923076923, 0.09775641025641028, 2.1862980769230766, 1.9535256410256412, 'face'] to be in the range [0.0, 1.0], got 1.0204326923076923.
Expected x_min for bbox [1.0709134615384615, 0.5625, 1.8040865384615383, 1.7644230769230769, 'face'] to be in the range [0.0, 1.0], got 1.0709134615384615.
Expected x_min for bbox [1.2223557692307692, 0.07852564102564102, 2.205528846153846, 1.767628205128205, 'face'] to be in the range [0.0, 1.0], got 1.2223557692307692.
Expected x_max for bbox [0.9915865384615385, 0.01762820512820514, 2.2175480769230766, 2.2451923076923075, 'face'] to be in the range [0.0, 1.0], got 2.2175480769230766.
Expected x_min for bbox [1.0901442307692308, 0.1169871794871795, 1.9483173076923077, 1.7964743589743588, 'face'] to be in the range [0.0, 1.0], got 1.0901442307692308.
Expec

In [18]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [19]:
train_images = tf.data.Dataset.list_files('aug_data/train/images/*.jpg', shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [22]:
test_images = tf.data.Dataset.list_files('aug_data/test/images/*.jpg', shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [23]:
val_images = tf.data.Dataset.list_files('aug_data/val/images/*.jpg', shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']